## Calculation of MSE

This notebook serves to calculate MSE across all treatments in Tahoe

In [3]:
import scanpy as sc
import pertpy as pt
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import scipy.stats as stats

## pseudobulk to cell_line and treatment only
from adpbulk import ADPBulk

In [2]:
pb_path = "/home/ubuntu/frameshift-1/data/20250213.Tahoe.merged.pseudobulk.public.h5ad"
# adata = sc.read_h5ad(pb_path, backed='r')
adata = sc.read_h5ad(pb_path)
adata

AnnData object with n_obs × n_vars = 67018 × 62710
    obs: 'sample', 'drugname_drugconc', 'drug', 'n_cells', 'tscp_count', 'plate', 'Cell_Name_Vevo', 'Cell_ID_Cellosaur'
    var: 'gene_id', 'genome'

In [4]:
# initialize the object
adpb = ADPBulk(adata, ["Cell_Name_Vevo", "drugname_drugconc"])

# perform the pseudobulking
pseudobulk_matrix = adpb.fit_transform()

# retrieve the sample meta data (useful for easy incorporation with edgeR)
sample_meta = adpb.get_meta()

pb = sc.AnnData(X=pseudobulk_matrix, obs=sample_meta.set_index('SampleName'))
pb

Aggregating Samples: 100%|██████████| 56900/56900 [00:30<00:00, 1863.03it/s]


AnnData object with n_obs × n_vars = 56877 × 62710
    obs: 'Cell_Name_Vevo', 'drugname_drugconc'

In [5]:
sc.pp.normalize_total(pb, target_sum=1e6, exclude_highly_expressed=True)
sc.pp.log1p(pb)

In [ ]:
# distance = pt.tl.Distance("mse", groupby)
# df = distance.pairwise(adata, groupby=obs_key)

In [6]:
mse = {}

gp = pb.obs.groupby('Cell_Name_Vevo')

control = "[('DMSO_TF', 0.0, 'uM')]"
for name, group in gp:
	control_idx = group[group['drugname_drugconc'] == control].index
	control_vec = pb[control_idx].X
	if control_vec.shape[0] != 1:
		print(group[group['drugname_drugconc'] == control])
	else:
		#get MSE values
		drug_idx = group[group['drugname_drugconc'] != control].index
		drug_vec = pb[drug_idx].X
		drug_names = group.loc[drug_idx, 'drugname_drugconc'].values
		#compute MSE
		mse_matr = drug_vec - control_vec
		mse_matr = np.square(mse_matr)
		mse_matr = np.mean(mse_matr, axis=1)
		mse[name] = pd.Series(mse_matr, index=drug_names)


	# control_idx = group[group['drugname_drugconc'] == control].index
	# if len(control_idx) == 0:
	# 	continue
	# for drug in group['drugname_drugconc'].unique():
	# 	if drug == control:
	# 		continue
	# 	#compute MSE for all treatments against DMSO
	# 	drug_idx

	# subdata = adata[group.index]

	# #compute MSE for all treatments against DMSO
	
	# break
msedf = pd.DataFrame(mse).T
msedf.to_csv("/home/ubuntu/frameshift-1/data/MSE.tsv", sep="\t", header=True, index=True)
msedf.head()

In [ ]:
msedf.median(axis=0).sort_values(ascending=False).head(20)